In [9]:
import scipy.io as scio  
from skimage import io
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import math
import glob

In [10]:
def cell_cut(folders_outline,folders_annatated,folders_raw):
    img_list_outline= os.listdir('./'+folders_outline)
    img_list_annatated= os.listdir('./'+folders_annatated)
    img_list_raw= os.listdir('./'+folders_raw)
    BLACK=[0,0,0]
    for m in range(len(img_list_annatated)):
        data = scio.loadmat('./'+folders_outline+'/'+img_list_outline[m])
        data = data['single_mask']
        img = cv2.imread('./'+folders_annatated + '/' + os.path.splitext(img_list_outline[m])[0]+'.png')
        img_raw = cv2.imread('./'+folders_raw + '/' + os.path.splitext(img_list_outline[m])[0]+'.png')
        b,g,r = cv2.split(img_raw)
        k=0
        for n in range(data.shape[2]):
            outline=data[:,:,n]
            ret,thresh = cv2.threshold(outline,0,255,0)
            b1 = np.multiply(b,outline)
            g1 = np.multiply(g,outline)
            r1 = np.multiply(r,outline)
            img_raw = cv2.merge([b1,g1,r1])
            image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            x,y,w,h = cv2.boundingRect(contours[0])
            slide = max(w,h)
            #x1 = x+slide
            #y1 = y+slide
            M = cv2.moments(contours[0])
            try:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            except ZeroDivisionError:
                continue
            else:
                x1 = cx-math.ceil(slide/2)
                y1 = cy-math.ceil(slide/2)
                x2 = cx+math.ceil(slide/2)
                y2 = cy+math.ceil(slide/2)
                if (cx<math.ceil(slide/2))or(cy<math.ceil(slide/2))or(cx>(513-math.ceil(slide/2)))or(cy>(513-math.ceil(slide/2))):
                    continue
                cropImg = img[y1:y2, x1:x2]
                HSV = cv2.cvtColor(cropImg, cv2.COLOR_BGR2HSV)
                green = yellow = red = 0
                try:
                    row_num, col_num= HSV.shape[:2]
                except AttributeError:
                    continue
                card_img_count = row_num * col_num
                for i in range(row_num):
                    for j in range(col_num):
                        H = HSV.item(i, j, 0)
                        S = HSV.item(i, j, 1)
                        V = HSV.item(i, j, 2)
                        if 11 < H <= 34 and S > 34:#图片分辨率调整
                            yellow += 1
                        elif 35 < H <= 99 and S > 34:#图片分辨率调整
                            green += 1
                        elif 177 < H <= 180 and S > 34:#图片分辨率调整
                            red += 1
                if (yellow<10)and(green<10)and(red<10):
                    continue
                if (yellow >= green)and(yellow >= red):
                    dir_name='./breast_padding/plasma cell'
                elif (green >= yellow)and(green >= red):
                    dir_name='./breast_padding/cancer cell'
                elif (red >= yellow)and(red >= green):
                    dir_name='./breast_padding/lymphocyte'
                cropImg = img_raw[y1:y2, x1:x2]
                border=35-math.ceil(slide/2)
                if border>0:
                    cropImg = cv2.copyMakeBorder(cropImg,border,border,border,border,cv2.BORDER_CONSTANT,value=BLACK)
                if not os.path.exists(dir_name):os.makedirs(dir_name)
                cv2.imwrite(dir_name+'/'+os.path.splitext(img_list_annatated[m])[0]+'_'+str(k)+'.png',cropImg)
                k = k+1
        #dir_name = './breast_raw/index'
        #if not os.path.exists(dir_name):os.makedirs(dir_name)
        #cv2.imwrite(dir_name+'/'+str(img_list_outline[m])+'_.png',img)

In [11]:
if __name__ == '__main__':
    cell_cut("single_mask1/","Cut Patches/breast/annotated/","Cut Patches/breast/Raw/")

In [12]:
def index_show(folders_outline,folders_annatated,slide):

    img_list_outline= os.listdir('./'+folders_outline)
    img_list_annatated= os.listdir('./'+folders_annatated)
    for m in range(len(img_list_annatated)):
        data = scio.loadmat('./'+folders_outline+'/'+img_list_outline[m])
        data = data['single_mask']
        img = cv2.imread('./'+folders_annatated + '/' + os.path.splitext(img_list_outline[m])[0]+'.png')
        k=0
        for n in range(data.shape[2]):
            ret,thresh = cv2.threshold(data[:,:,n],0,255,0)
            image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            x,y,w,h = cv2.boundingRect(contours[0])
            slide = max(w,h)
            #x1 = x+slide
            #y1 = y+slide
            M = cv2.moments(contours[0])
            try:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            except ZeroDivisionError:
                continue
            else:
                x1 = cx-math.ceil(slide/2)
                y1 = cy-math.ceil(slide/2)
                x2 = cx+math.ceil(slide/2)
                y2 = cy+math.ceil(slide/2)
                if (cx<math.ceil(slide/2))or(cy<math.ceil(slide/2))or(cx>(513-math.ceil(slide/2)))or(cy>(513-math.ceil(slide/2))):
                    continue
                cropImg = img[y1:y2, x1:x2]
                HSV = cv2.cvtColor(cropImg, cv2.COLOR_BGR2HSV)
                green = yellow = red = 0
                try:
                    row_num, col_num= HSV.shape[:2]
                except AttributeError:
                    continue
                card_img_count = row_num * col_num
                for i in range(row_num):
                    for j in range(col_num):
                        H = HSV.item(i, j, 0)
                        S = HSV.item(i, j, 1)
                        V = HSV.item(i, j, 2)
                        if 11 < H <= 34 and S > 34:#图片分辨率调整
                            yellow += 1
                        elif 35 < H <= 99 and S > 34:#图片分辨率调整
                            green += 1
                        elif 177 < H <= 180 and S > 34:#图片分辨率调整
                            red += 1
                if (yellow<10)and(green<10)and(red<10):
                    continue
                if (yellow >= green)and(yellow >= red):
                    #cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,255),1)
                    cv2.drawContours(img,contours,0,(0,255,255),1)
                    font=cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img,str(k),(cx,cy), font, 0.4,(0,255,255),1)
                elif (green >= yellow)and(green >= red):
                    #cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
                    cv2.drawContours(img,contours,0,(0,255,0),1)
                    font=cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img,str(k),(cx,cy), font, 0.4,(0,255,0),1)
                elif (red >= yellow)and(red >= green):
                    #cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)
                    cv2.drawContours(img,contours,0,(0,0,255),1)
                    font=cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(img,str(k),(cx,cy), font, 0.4,(0,0,255),1)
            #cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
            #font=cv2.FONT_HERSHEY_SIMPLEX
            #cv2.putText(img,str(k),(cx,cy), font, 0.4,(255,255,255),1)
                k = k+1
        dir_name='./breast_padding/index'
        if not os.path.exists(dir_name):os.makedirs(dir_name)
        cv2.imwrite(dir_name+'/'+os.path.splitext(img_list_annatated[m])[0]+'.png',img)

In [13]:
if __name__ == '__main__':
    index_show("single_mask1/","Cut Patches/breast/annotated/",70)

In [1]:
def box_show(folders_outline,folders_annatated,slide):
    folders_outline="single_mask1/"
    folders_annatated="Cut Patches/breast/annotated/"
    img_list_outline= os.listdir('./'+folders_outline)
    img_list_annatated= os.listdir('./'+folders_annatated)
    for m in range(len(img_list_annatated)):
        data = scio.loadmat('./'+folders_outline+'/'+img_list_outline[m])
        data = data['single_mask']
        img = cv2.imread('./'+folders_annatated + '/' + img_list_annatated[m])
        k=1
        for n in range(data.shape[2]):
            ret,thresh = cv2.threshold(data[:,:,n],0,255,0)
            image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            x,y,w,h = cv2.boundingRect(contours[0])
            slide = max(w,h)
            #x1 = x+slide
            #y1 = y+slide
            M = cv2.moments(contours[0])
            try:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
            except ZeroDivisionError:
                continue
            else:
                x1 = cx-math.ceil(slide/2)
                y1 = cy-math.ceil(slide/2)
                x2 = cx+math.ceil(slide/2)
                y2 = cy+math.ceil(slide/2)
                if (cx<math.ceil(slide/2))or(cy<math.ceil(slide/2))or(cx>(513-math.ceil(slide/2)))or(cy>(513-math.ceil(slide/2))):
                    continue
                cropImg = img[y1:y2, x1:x2]
                HSV = cv2.cvtColor(cropImg, cv2.COLOR_BGR2HSV)
                green = yellow = red = 0
                try:
                    row_num, col_num= HSV.shape[:2]
                except AttributeError:
                    continue
                card_img_count = row_num * col_num
                for i in range(row_num):
                    for j in range(col_num):
                        H = HSV.item(i, j, 0)
                        S = HSV.item(i, j, 1)
                        V = HSV.item(i, j, 2)
                        if 11 < H <= 34 and S > 34:#图片分辨率调整
                            yellow += 1
                        elif 35 < H <= 99 and S > 34:#图片分辨率调整
                            green += 1
                        elif 177 < H <= 180 and S > 34:#图片分辨率调整
                            red += 1
                if (yellow<10)and(green<10)and(red<10):
                    continue
                if (yellow >= green)and(yellow >= red):
                    own_slide=50
                    x1 = cx-math.ceil(own_slide/2)
                    y1 = cy-math.ceil(own_slide/2)
                    x2 = cx+math.ceil(own_slide/2)
                    y2 = cy+math.ceil(own_slide/2)
                    cv2.rectangle(img,(x1,y1),(x2,y2),(255,255,255),1)
                    #cv2.drawContours(img,contours[0][n],(0,255,255),1)
                elif (green >= yellow)and(green >= red):
                    own_slide=64
                    x1 = cx-math.ceil(own_slide/2)
                    y1 = cy-math.ceil(own_slide/2)
                    x2 = cx+math.ceil(own_slide/2)
                    y2 = cy+math.ceil(own_slide/2)
                    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
                    #cv2.drawContours(img,contours[0][n],(0,255,0),1)
                elif (red >= yellow)and(red >= green):
                    own_slide=30
                    x1 = cx-math.ceil(own_slide/2)
                    y1 = cy-math.ceil(own_slide/2)
                    x2 = cx+math.ceil(own_slide/2)
                    y2 = cy+math.ceil(own_slide/2)
                    cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),1)
                    #cv2.drawContours(img,contours[0][n],(255,0,0),1)
            #cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
                font=cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(img,str(k),(cx,cy), font, 0.4,(255,255,255),1)
                k = k+1
        dir_name='./breast_raw/index'
        if not os.path.exists(dir_name):os.makedirs(dir_name)
        cv2.imwrite(dir_name+'/'+os.path.splitext(img_list_annatated[m])[0]+'.png',img)

In [4]:
if __name__ == '__main__':
    box_show("single_mask1/","Cut Patches/breast/annotated/",60)
    '''注意边框的颜色可能影响区域里面颜色的判断，上面的绿色应该还需要修改'''